**Instructions**:

1. Go to the *Cell* menu and click *Run All*

#### Code setup

In [0]:
# Need to force these installs on Binder
!pip install nbformat
!pip install numpy
!pip install matplotlib

In [0]:
from collections import deque, Counter
import string
import itertools
import time

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import ipywidgets as widgets

In [20]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [0]:
plt.rcParams['figure.figsize'] = (18, 9)

In [0]:
def caesar(frequencies, k):
    res = frequencies.copy()
    res.rotate(k)
    return res

In [0]:
def decrypt_text(ct, k):
    ct_letters = string.ascii_uppercase
    pt_letters = deque(ct_letters)
    pt_letters.rotate(k)
    decrypt_map = dict(zip(ct_letters, pt_letters))
    return ''.join((decrypt_map.get(c, c) for c in ct))

In [0]:
def circular_bar_plot(labels, heights,
                      bottom=1, max_height=4,
                      clockwise=True, title=None, color='blue',
                      ax=None, letter_fontsize=14, number_fontsize=12,
                      title_fontsize=18):
    assert len(heights) == len(labels)
    N = len(labels)

    if ax is None:
        ax = plt.subplot(111, polar=True)
    
    if clockwise:
        raw_thetas = np.linspace(1/2 * np.pi, -3/2 * np.pi, N, endpoint=False)
    else:
        raw_thetas = np.linspace(1/2 * np.pi, 5/2 * np.pi, N, endpoint=False)

    # Matplotlib requires the θ-coords to be in the range [0, 2π)
    thetas = np.mod(raw_thetas, 2*np.pi)

    height_scaler = np.vectorize(lambda h: max_height * h / max(heights))
    scaled_heights = height_scaler(heights)
    yticks_raw = np.arange(0, max(heights), 0.05)
    yticks = height_scaler(yticks_raw) + bottom
    yticklabels = [f'{h*100:.0f}%' for h in yticks_raw]

    bars = ax.bar(thetas, scaled_heights, width=2*np.pi / N, bottom=bottom,
                  color=color)

    ax.set_xticks(thetas)
    ax.set_xticklabels(labels, fontsize=letter_fontsize)
    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels, fontsize=number_fontsize)

    if title:
        ax.set_title(title, fontsize=title_fontsize)

In [0]:
# from: https://en.wikipedia.org/wiki/Letter_frequency#Relative_frequencies_of_the_first_letters_of_a_word_in_the_English_language
letter_frequency_raw = '''0.08167
0.01492
0.02782
0.04253
0.12702
0.02228
0.02015
0.06094
0.06966
0.00153
0.00772
0.04025
0.02406
0.06749
0.07507
0.01929
0.00095
0.05987
0.06327
0.09056
0.02758
0.00978
0.02360
0.00150
0.01974
0.00074'''


In [0]:
letter_frequencies = deque([float(x) for x in letter_frequency_raw.splitlines()])
letters = deque(string.ascii_uppercase)

In [0]:
def compare(k):
    axes = [plt.subplot(1, 2, i+1, polar=True) for i in range(2)]

    circular_bar_plot(letters, letter_frequencies, ax=axes[0],
                    title='Average English', color='#070')
    circular_bar_plot(letters, caesar(letter_frequencies, k), ax=axes[1],
                    title=f'Average English (Caesar encrypted with key {k})',
                    color='#707')
    clear_output(wait=True)
    plt.show()

In [0]:
def compare_output(k, op):
    # Output to the ipywidgets.Output op
    with op:
        compare(k)

In [0]:
def frequencies_from_text(text):
    assert isinstance(text, str)
    all_counts = Counter(text.upper())
    letter_counts = [all_counts.get(c, 0) for c in string.ascii_uppercase]
    return deque([(x/ sum(letter_counts)) for x in letter_counts])

In [0]:
def try_decrypt(text_frequencies, k):
    axes = [plt.subplot(1, 2, i+1, polar=True) for i in range(2)]

    circular_bar_plot(letters, letter_frequencies, ax=axes[0],
                    title='Average English', color='#070')
    circular_bar_plot(letters, caesar(text_frequencies, -k), ax=axes[1],
                    title=f'Ciphertext (Caesar decrypted with key {k})',
                    color='#007')
    clear_output(wait=True)
    plt.show()

In [0]:
def try_decrypt_output(text_frequencies, k, op):
    with op:
        try_decrypt(text_frequencies, k)

In [0]:
def interactive_average_frequencies():
    slider = widgets.IntSlider(value=1, min=1, max=25, step=1,
                           description='Key', continuous_update=False)
    out = widgets.Output()

    update_fn = lambda _: compare_output(slider.value, out)
    slider.observe(update_fn, names='value')

    box = widgets.VBox([slider, out])
    update_fn(None)
    display(box)

In [0]:
def interactive_ciphertext_analysis(ct):
    ct_frequencies = frequencies_from_text(ct)
    slider = widgets.IntSlider(value=1, min=1, max=25, step=1,
                            description='Key', continuous_update=False)
    out1 = widgets.Output()

    update_fn = lambda _: try_decrypt_output(ct_frequencies, slider.value, out1)
    slider.observe(update_fn, names='value')

    button = widgets.Button(description='Check decryption result',
                            layout=widgets.Layout(min_width='200px'))
    out2 = widgets.Output()

    def show():
        with out2:
            out2.clear_output()
            print(decrypt_text(ct, slider.value))

    button.on_click(lambda _: show())

    box = widgets.VBox([slider, out1, button, out2])
    update_fn(None)
    display(box)

#### How frequent are letters in unencrypted and encrypted English?

**Move the "Key" slider to see the letter frequencies we expect in encrypted text...**

In [34]:
interactive_average_frequencies()

#### Can we guess the key for a ciphertext this way?

In [0]:
ct = '''HGX TIIKHTVA MH IKXOXGM LNVA TMMTVDL BGOHEOXL MAX NLX HY T INUEBV DXR BGYKTLMKNVMNKX (IDB); T LXM HY KHEXL, IHEBVBXL, TGW IKHVXWNKXL GXXWXW MH VKXTMX, FTGTZX, WBLMKBUNMX, NLX, LMHKX TGW KXOHDX WBZBMTE VXKMBYBVTMXL TGW FTGTZX INUEBV-DXR XGVKRIMBHG. AHPXOXK, MABL BG MNKG ATL IHMXGMBTE PXTDGXLLXL.

YHK XQTFIEX, MAX VXKMBYBVTMX TNMAHKBMR BLLNBGZ MAX VXKMBYBVTMX FNLM UX MKNLMXW MH ATOX IKHIXKER VAXVDXW MAX BWXGMBMR HY MAX DXR-AHEWXK, FNLM XGLNKX MAX VHKKXVMGXLL HY MAX INUEBV DXR PAXG BM BLLNXL T VXKMBYBVTMX, FNLM UX LXVNKX YKHF VHFINMXK IBKTVR, TGW FNLM ATOX FTWX TKKTGZXFXGML PBMA TEE ITKMBVBITGML MH VAXVD TEE MAXBK VXKMBYBVTMXL UXYHKX IKHMXVMXW VHFFNGBVTMBHGL VTG UXZBG. PXU UKHPLXKL, YHK BGLMTGVX, TKX LNIIEBXW PBMA T EHGZ EBLM HY "LXEY-LBZGXW BWXGMBMR VXKMBYBVTMXL" YKHF IDB IKHOBWXKL – MAXLX TKX NLXW MH VAXVD MAX UHGT YBWXL HY MAX VXKMBYBVTMX TNMAHKBMR TGW MAXG, BG T LXVHGW LMXI, MAX VXKMBYBVTMXL HY IHMXGMBTE VHFFNGBVTMHKL. TG TMMTVDXK PAH VHNEW LNUOXKM TGR LBGZEX HGX HY MAHLX VXKMBYBVTMX TNMAHKBMBXL BGMH BLLNBGZ T VXKMBYBVTMX YHK T UHZNL INUEBV DXR VHNEW MAXG FHNGM T "FTG-BG-MAX-FBWWEX" TMMTVD TL XTLBER TL BY MAX VXKMBYBVTMX LVAXFX PXKX GHM NLXW TM TEE. BG TG TEMXKGTMX LVXGTKBH KTKXER WBLVNLLXW[VBMTMBHG GXXWXW], TG TMMTVDXK PAH IXGXMKTMXL TG TNMAHKBMR'L LXKOXKL TGW HUMTBGL BML LMHKX HY VXKMBYBVTMXL TGW DXRL (INUEBV TGW IKBOTMX) PHNEW UX TUEX MH LIHHY, FTLJNXKTWX, WXVKRIM, TGW YHKZX MKTGLTVMBHGL PBMAHNM EBFBM.

WXLIBMX BML MAXHKXMBVTE TGW IHMXGMBTE IKHUEXFL, MABL TIIKHTVA BL PBWXER NLXW. XQTFIEXL BGVENWX MEL TGW BML IKXWXVXLLHK LLE, PABVA TKX VHFFHGER NLXW MH IKHOBWX LXVNKBMR YHK PXU UKHPLXK MKTGLTVMBHGL (YHK XQTFIEX, MH LXVNKXER LXGW VKXWBM VTKW WXMTBEL MH TG HGEBGX LMHKX).

TLBWX YKHF MAX KXLBLMTGVX MH TMMTVD HY T ITKMBVNETK DXR ITBK, MAX LXVNKBMR HY MAX VXKMBYBVTMBHG ABXKTKVAR FNLM UX VHGLBWXKXW PAXG WXIEHRBGZ INUEBV DXR LRLMXFL. LHFX VXKMBYBVTMX TNMAHKBMR – NLNTEER T INKIHLX-UNBEM IKHZKTF KNGGBGZ HG T LXKOXK VHFINMXK – OHNVAXL YHK MAX BWXGMBMBXL TLLBZGXW MH LIXVBYBV IKBOTMX DXRL UR IKHWNVBGZ T WBZBMTE VXKMBYBVTMX. INUEBV DXR WBZBMTE VXKMBYBVTMXL TKX MRIBVTEER OTEBW YHK LXOXKTE RXTKL TM T MBFX, LH MAX TLLHVBTMXW IKBOTMX DXRL FNLM UX AXEW LXVNKXER HOXK MATM MBFX. PAXG T IKBOTMX DXR NLXW YHK VXKMBYBVTMX VKXTMBHG ABZAXK BG MAX IDB LXKOXK ABXKTKVAR BL VHFIKHFBLXW, HK TVVBWXGMTEER WBLVEHLXW, MAXG T "FTG-BG-MAX-FBWWEX TMMTVD" BL IHLLBUEX, FTDBGZ TGR LNUHKWBGTMX VXKMBYBVTMX PAHEER BGLXVNKX.'''

**Move the "Key" slider and see how the frequencies change in the decrypted text.  Click "Check decryption result" to test your guess of the key.**

In [36]:
interactive_ciphertext_analysis(ct)